In [7]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

In [8]:
base = pd.read_csv('../games.csv')

In [9]:
base = base.drop('NA_Sales', axis = 1)
base = base.drop('Developer', axis = 1)
base = base.drop('EU_Sales', axis = 1)
base = base.drop('JP_Sales', axis = 1)
base = base.drop('Other_Sales', axis = 1)

In [10]:
base = base.dropna(axis = 0)
base = base.loc[base['Global_Sales'] > 1]
names = base.Name
base = base.drop('Name', axis = 1)

In [11]:
base

,Platform,Year_of_Release,Genre,Publisher,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,82.53,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,35.52,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,32.77,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,29.80,89.0,65.0,8.5,431.0,E
7,Wii,2006.0,Misc,Nintendo,28.92,58.0,41.0,6.6,129.0,E
...,...,...,...,...,...,...,...,...,...,...
2050,X360,2014.0,Action,Ubisoft,1.01,72.0,32.0,7.3,318.0,M
2051,PS4,2014.0,Platform,Ubisoft,1.01,90.0,16.0,8.5,525.0,E10+
2053,X360,2008.0,Action,Ubisoft,1.01,81.0,73.0,7,189.0,T
2055,WiiU,2012.0,Action,Ubisoft,1.01,77.0,70.0,7.7,758.0,M


In [12]:
previsors = base.iloc[:, [0, 1, 2, 3, 5, 6, 7, 8, 9]].values
Global_sales = base.iloc[:, 4].values

In [14]:
previsors

array([[12, 2006.0, 10, ..., '8', 322.0, 'E'],
       [12, 2008.0, 6, ..., '8.3', 709.0, 'E'],
       [12, 2009.0, 10, ..., '8', 192.0, 'E'],
       ...,
       [14, 2008.0, 0, ..., '7', 189.0, 'T'],
       [13, 2012.0, 0, ..., '7.7', 758.0, 'M'],
       [13, 2016.0, 2, ..., '8', 273.0, 'E10+']], dtype=object)

In [15]:
labelEncoder_prev = LabelEncoder()
previsors[:,0] = labelEncoder_prev.fit_transform(previsors[:,0])
previsors[:,2] = labelEncoder_prev.fit_transform(previsors[:,2])
previsors[:,3] = labelEncoder_prev.fit_transform(previsors[:,3])
previsors[:,8] = labelEncoder_prev.fit_transform(previsors[:,8])

In [17]:
previsors

array([[12, 2006.0, 10, ..., '8', 322.0, 1],
       [12, 2008.0, 6, ..., '8.3', 709.0, 1],
       [12, 2009.0, 10, ..., '8', 192.0, 1],
       ...,
       [14, 2008.0, 0, ..., '7', 189.0, 5],
       [13, 2012.0, 0, ..., '7.7', 758.0, 4],
       [13, 2016.0, 2, ..., '8', 273.0, 2]], dtype=object)

In [18]:
encoder = OneHotEncoder(sparse=False)
cols_to_encode = [0,2,3,8]
columns_to_encode = previsors[:, cols_to_encode]
encoded_columns = encoder.fit_transform(columns_to_encode)
dataset_encoded = np.delete(previsors, cols_to_encode, axis=1)
dataset_encoded = np.concatenate((dataset_encoded, encoded_columns), axis=1)
previsorsEncoded = dataset_encoded.astype(float)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [22]:
inputLayer = Input(shape=(99,))
sigmoid = Activation('sigmoid')
linear = Activation('linear')
hiddenLayer1 = Dense(units=45, activation=sigmoid)(inputLayer)
hiddenLayer2 = Dense(units=45, activation=sigmoid)(hiddenLayer1)
outputLayer1 = Dense(units=1, activation=linear)(hiddenLayer2)

In [23]:
regressor = Model(inputs=inputLayer, outputs=[outputLayer1])
regressor.compile(optimizer='adam', loss='mse')

In [24]:
regressor.fit(previsorsEncoded, [Global_sales], epochs=5000, batch_size=100)

Epoch 1/5000
14/14 [==============================] - 1s 2ms/step - loss: 20.2762
Epoch 2/5000
14/14 [==============================] - 0s 2ms/step - loss: 17.3660
Epoch 3/5000
14/14 [==============================] - 0s 2ms/step - loss: 15.6084
Epoch 4/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.8156
Epoch 5/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.4559
Epoch 6/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.3497
Epoch 7/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.2835
Epoch 8/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.2143
Epoch 9/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.1708
Epoch 10/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.1363
Epoch 11/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.0835
Epoch 12/5000
14/14 [==============================] - 0s 2ms/step - loss: 14.0380
Epoch 13/5000